In [2]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO
import cv2


In [2]:

# Ver versión de PyTorch
print(f"Versión de PyTorch: {torch.__version__}")

# Ver disponibilidad de CUDA
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA no está disponible.")


Versión de PyTorch: 2.5.1+cu124
CUDA disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
# Configuración del dispositivo (GPU o CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parámetros del modelo y entrenamiento
NUM_CLASSES = 10  # Número de equipos de F1
LEARNING_RATE = 0.0005
BATCH_SIZE = 8  # Ajusta según la memoria de la GPU
NUM_WORKERS = 8  # Aumentar si tienes más núcleos en la CPU
NUM_EPOCHS = 50 # O más, dependiendo del rendimiento
AUGMENTATION = True  # Hacer augmentación de datos




In [4]:
import cv2
import numpy as np

# Verificar si las imágenes se leen correctamente
train_image = cv2.imread("../f1-strategy/f1-dataset/train/images/1-2023-Brazilian-GP-FP1-20_jpg.rf.2f3d25867c8e1661f6070ae5a84c6dd4.jpg")
if train_image is None:
    print("Error al leer la imagen")
else:
    print("Imagen cargada correctamente: ", train_image.shape)


Imagen cargada correctamente:  (640, 640, 3)


In [5]:
import cv2
import os

# Definir la carpeta de imágenes
image_folder = "C:/Users/victo/Desktop/Documents/Tercer año/Segundo Cuatrimestre/Finales/f1-strategy/f1-dataset/train/images"

# Lista para almacenar los tamaños de las imágenes
sizes = set()

# Verificar el tamaño de cada imagen
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    img = cv2.imread(image_path)
    
    if img is not None:
        img_size = img.shape  # (height, width, channels)
        sizes.add(img_size)
    else:
        print(f"Error al cargar la imagen: {image_name}")

# Comprobar si todas las imágenes tienen el mismo tamaño
if len(sizes) == 1:
    print("Todas las imágenes tienen el mismo tamaño.")
else:
    print("Hay imágenes con tamaños diferentes. Los tamaños encontrados son:")
    for size in sizes:
        print(size)


Todas las imágenes tienen el mismo tamaño.


In [6]:
# Cargar el modelo YOLOv8 Nano
from torch.optim import Adam
model = YOLO("yolov8m.pt")



model.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=NUM_EPOCHS,
    batch=BATCH_SIZE,
    optimizer="SGD",         # Puedes probar también con "AdamW" para un optimizador adaptativo
    lr0=LEARNING_RATE,
    momentum=0.9,
    warmup_epochs=5,         # Menos epochs de warmup para entrar más rápido en el entrenamiento efectivo
    weight_decay=0.0001,
    label_smoothing=0.1,
    degrees=15.0,            # Rotación máxima reducida a 15° para evitar transformaciones demasiado agresivas
    mixup=0.15,              # Mezcla de imágenes de forma moderada
    copy_paste=0.05,         # Menor intensidad para no alterar en exceso la composición real de la imagen
    hsv_h=0.015,             # Cambios sutiles en el tono para mantener la naturalidad
    box=7.5,
    cls=1.5,
    patience=10,             # Más paciencia para que el scheduler pueda adaptarse y reducir la lr si es necesario
    pretrained=True,
    project='../f1-strategy/yolo-files/runs',
    name='train_v1',
)













100%|██████████| 49.7M/49.7M [00:04<00:00, 12.3MB/s]


New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=50, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../f1-strategy/yolo-files/runs, name=train_v1, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream

100%|██████████| 5.35M/5.35M [00:00<00:00, 10.3MB/s]


AMP: checks passed 


train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:01<00:00, 2207.81it/s]


...

                   all         20         29      0.592      0.653      0.724      0.613
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 37, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



47 epochs completed in 0.862 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_v1\weights\last.pt, 52.0MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_v1\weights\best.pt, 52.0MB

Validating ..\f1-strategy\yolo-files\runs\train_v1\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.03it/s]


                   all         20         29      0.639       0.72      0.763      0.634
           Kick Sauber          4          6      0.786        0.5      0.691      0.557
          Racing Bulls          1          1      0.354          1      0.995      0.796
                Alpine          4          5      0.532        0.6      0.669      0.567
               Ferarri          3          4      0.689       0.75      0.812      0.702
                  Haas          1          1      0.589          1      0.995      0.895
               Mclaren          5          6          1       0.58      0.894      0.742
              Mercedes          3          3      0.842      0.333       0.72      0.544
              Williams          2          3      0.316          1      0.332       0.27
Speed: 0.2ms preprocess, 8.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_v1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022CAB7E5060>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

# Resultados con 200 imágenes
### Model With this paremeters and no optimizer:
#### Configuración del dispositivo (GPU o CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Parámetros del modelo y entrenamiento
NUM_CLASSES = 10  # Número de equipos de F1
BATCH_SIZE = 8
NUM_WORKERS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 0.005

Mal rendimiento, pasando a nuevos valores y reentrenamiento.

### Aun mal rendimiento, Pasando a un modelo mas complejo, del nano al m.

### Nuevo modelo tras data augmentation
Después de aumentar las imágenes a 200 por clase, rendimiento del 70% obtenido en todas las clases. 
Sí, recuerdo los resultados anteriores que compartiste. Por ejemplo, tenías:

Métricas generales:

mAP50: 0.772
mAP50-95: 0.449
Precisión (P): 0.733
Recall (R): 0.615
Clases problemáticas y sus mAP50:

Williams: 0.487
Kick Sauber: 0.619 (con un recall notablemente bajo, alrededor de 0.443)
Alpine: 0.556
Clases con buenos resultados:

Racing Bulls y Haas: mAP50 > 0.99
Mclaren y Mercedes: mAP50 > 0.88
Tiempo de inferencia: 8.1 ms por imagen

Williams y Sauber con muchos falsos positivos, por lo que:

- Probamos con nuevos hiperparámetros y aumentamos las epocas.






In [7]:
model.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.11s/it]


                   all         20         29      0.638      0.721      0.762      0.633
           Kick Sauber          4          6      0.784        0.5      0.691      0.557
          Racing Bulls          1          1      0.353          1      0.995      0.796
                Alpine          4          5      0.532        0.6      0.669      0.567
               Ferarri          3          4      0.689       0.75      0.812      0.702
                  Haas          1          1      0.588          1      0.995      0.895
               Mclaren          5          6          1      0.581      0.887      0.735
              Mercedes          3          3      0.841      0.333       0.72      0.544
              Williams          2          3      0.316          1      0.332       0.27
Speed: 2.1ms preprocess, 35.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_v12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022CABCA8CA0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

Comparación y Conclusión
Mejora Global:
La nueva configuración muestra una mejora en el mAP50 (0.818 vs 0.772) y en el mAP50-95 (0.552 vs 0.449), lo que indica que el modelo está detectando con mayor precisión y que sus cajas se ajustan mejor a los objetos reales.

Precisión:
La precisión global ha subido de 0.733 a 0.765, lo que es un buen indicativo de que las predicciones correctas han aumentado.

Recall:
El recall se mantiene prácticamente igual (0.615 vs 0.61), lo que sugiere que el número de falsos negativos no varió significativamente.
Sin embargo, la mejora en mAP50-95 implica que las detecciones precisas han mejorado, aunque quizá se deba trabajar estrategias adicionales para las clases problemáticas (como Kick Sauber, Alpine y Williams).

Hiperparámetros:
El uso de una tasa de aprendizaje variable (con warmup y decaimiento), junto con ajustes más moderados en la intensidad de las transformaciones (reduciendo grados de rotación, mixup y copy_paste) parecen haber contribuido a esta mejora global.

Consideraciones:
Las clases con peor desempeño (por ejemplo, Williams) siguen siendo un reto. Podría ser interesante explorar aumentos específicos o técnicas de fine-tuning para esas clases en particular.

#### Proóximos pasos.

Aumentar a 50 epochs.
Hace fine tuning de las clases problematicas.

# Modelo v3: reentrenar para 250 imágenes

Resultados comparados con el anterior modelo de 200 imágenes: 


# Repetir fine tuning pero para v3


Resultados Globales
mAP50:

Anterior: 0.84
Nuevo: 0.871
→ Mejora global.
mAP50-95:

Anterior: 0.728
Nuevo: 0.752
→ También mejora ligeramente.
Por Clase
Kick Sauber:

Anterior: mAP50 = 0.714, mAP50-95 = 0.575
Nuevo: mAP50 = 0.849, mAP50-95 = 0.585
→ Se observa una mejora significativa en mAP50, lo cual es muy positivo.
Racing Bulls:

Ambos modelos mantienen mAP50 muy altos (0.995), con un leve aumento en mAP50-95 (de 0.796 a 0.895).
Alpine:

Anterior: mAP50 = 0.928, mAP50-95 = 0.876
Nuevo: mAP50 = 0.753, mAP50-95 = 0.71
→ Se degrada considerablemente el rendimiento, lo que sugiere que los nuevos parámetros (por ejemplo, una mayor rotación o mixup) podrían estar afectando negativamente a esta clase.
Ferarri:

Una ligera reducción en mAP50 (de 0.845 a 0.808), pero mAP50-95 se mantiene prácticamente igual.
Haas:

Permanece estable (mAP50 ≈ 0.995 y mAP50-95 ≈ 0.895).
Mclaren:

Mejora de mAP50 de 0.805 a 0.903 y un leve incremento en mAP50-95 (de 0.738 a 0.767).
Mercedes:

Se mantiene mAP50 en 0.913, con un aumento en mAP50-95 de 0.726 a 0.793.
Williams:

Mejora de mAP50 de 0.522 a 0.753 y mAP50-95 de 0.473 a 0.627
→ Es un resultado positivo, especialmente si en el YAML se ha bajado el peso de esta clase.
Conclusiones y Recomendaciones
Globalmente, el nuevo fine-tuned mejora las métricas generales (mAP50 y mAP50-95), lo que indica que el modelo, en promedio, está detectando y ajustando mejor las cajas.
Mejoras destacadas en clases problemáticas como Kick Sauber, Mclaren, Mercedes y Williams sugieren que aumentar el dataset a 250 imágenes y ajustar los parámetros (incluyendo la modificación de pesos en el YAML para Williams) ha beneficiado a estas clases.
La principal preocupación es Alpine, cuya precisión (mAP50) se redujo notablemente. Esto podría deberse a:
Los nuevos parámetros de augmentación (por ejemplo, grados de rotación de 15° y mixup de 0.3) podrían estar generando transformaciones que afectan negativamente las características de Alpine.
Es posible que la variabilidad en Alpine no se esté reflejando adecuadamente en el conjunto de datos.
Recomendación:

Es viable continuar con el fine tuning usando 250 imágenes por clase y ajustar el peso de Williams a 1.2 en el YAML, ya que esto ha mostrado mejoras para esa clase.
Sin embargo, sería conveniente revisar o ajustar las transformaciones específicas para Alpine (por ejemplo, disminuir la rotación o el mixup) para intentar recuperar parte del rendimiento perdido en esa clase.
En resumen, el nuevo fine tuning mejora el desempeño global y de varias clases problemáticas, aunque se requiere un ajuste focalizado para Alpine

In [11]:
model.save("../f1-strategy/weights/f1_detection_best.pt")

In [13]:
model_fine_tuned = YOLO("../f1-strategy/weights/f1_detection_best.pt")  

# model_fine_tuned.train(
#     data="../f1-strategy/f1-dataset/data.yaml",
#     epochs=NUM_EPOCHS,
#     batch=BATCH_SIZE,
#     optimizer="AdamW",
#     lr0=LEARNING_RATE / 2,  # Aprendizaje más lento para no sobreajustar
#     momentum=0.9,
#     weight_decay=0.0001,
#     label_smoothing=0.1,
#     degrees=10.0,
#     mixup=0.2,
#     patience=5,
#     pretrained=True,  # Importante para mantener el conocimiento previo
#     project='../f1-strategy/yolo-files/runs',
#     name='train_finetune'
# )


# model_fine_tuned.train(
#     data="../f1-strategy/f1-dataset/data.yaml",
#     epochs=NUM_EPOCHS,
#     batch=BATCH_SIZE,
#     optimizer="AdamW",
#     lr0=LEARNING_RATE / 2,  # Mantener un aprendizaje más lento para no sobreajustar
#     momentum=0.9,
#     weight_decay=0.0001,  # Mantener el ajuste original para evitar sobreajuste
#     label_smoothing=0.1,
#     degrees=10.0,
#     mixup=0.2,
#     patience=10,  # Paciencia ajustada a 10
#     pretrained=True,  # Mantener el conocimiento previo
#     project='../f1-strategy/yolo-files/runs',
#     name='train_finetune',
# )

model_fine_tuned.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=NUM_EPOCHS,
    batch=BATCH_SIZE,
    optimizer="AdamW",
    lr0=LEARNING_RATE / 3,  # Reducción más agresiva del learning rate
    momentum=0.9,
    weight_decay=0.001,  # Aumentar regularización
    label_smoothing=0.2,  # Mayor suavizado para evitar overconfidence
    degrees=15.0,  # Más rotación para ayudar con ángulos lejanos
    mixup=0.3,  # Mayor mezcla para generalización
    patience=15,  # Más paciencia para convergencia
    pretrained=True,
    project='../f1-strategy/yolo-files/runs',
    name='train_finetune',
    flipud=0.1,  # Añadir flip vertical
    shear=2.0  # Distorsión adicional
)


New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=../f1-strategy/weights/f1_detection_best.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=50, time=None, patience=15, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../f1-strategy/yolo-files/runs, name=train_finetune, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plot

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_finetune\labels.jpg... 
optimizer: AdamW(lr=0.00016666666666666666, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.001), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_finetune
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         29      0.677       0.65      0.782      0.692
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 15, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



30 epochs completed in 0.551 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_finetune\weights\last.pt, 52.0MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_finetune\weights\best.pt, 52.0MB

Validating ..\f1-strategy\yolo-files\runs\train_finetune\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.60it/s]


                   all         20         29      0.699      0.792      0.871      0.752
           Kick Sauber          4          6          1      0.461      0.849      0.585
          Racing Bulls          1          1      0.522          1      0.995      0.895
                Alpine          4          5      0.639      0.716      0.753       0.71
               Ferarri          3          4      0.615       0.75      0.808      0.742
                  Haas          1          1      0.403          1      0.995      0.895
               Mclaren          5          6      0.826      0.796      0.903      0.767
              Mercedes          3          3          1      0.614      0.913      0.793
              Williams          2          3      0.591          1      0.753      0.627
Speed: 0.6ms preprocess, 7.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022CABFEF6D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [14]:
# Evaluar después del fine-tuning
model_fine_tuned.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


                   all         20         29        0.7      0.792      0.871      0.752
           Kick Sauber          4          6          1      0.461      0.849      0.585
          Racing Bulls          1          1      0.521          1      0.995      0.895
                Alpine          4          5      0.638      0.715      0.753       0.71
               Ferarri          3          4      0.615       0.75      0.808      0.742
                  Haas          1          1      0.403          1      0.995      0.895
               Mclaren          5          6      0.826      0.797      0.903      0.767
              Mercedes          3          3          1      0.614      0.913      0.793
              Williams          2          3      0.594          1      0.753      0.627
Speed: 2.2ms preprocess, 33.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022CA7200BB0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [15]:
# Guardar el modelo fine-tuned con un nuevo nombre
model_fine_tuned.save("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")

print("Modelo fine-tuned guardado correctamente en: ../f1-strategy/weights/f1_detection_fine_tuned_best.pt")


Modelo fine-tuned guardado correctamente en: ../f1-strategy/weights/f1_detection_fine_tuned_best.pt


# Resultados con 200 imágenes

### Conclusiones del modelo fine tuned: Mejor usar el best o el best_fine_tuned?
Resultados del Modelo Anterior:
mAP50: 0.772
mAP50-95: 0.449
Precisión (P): 0.733
Recall (R): 0.615
Clases Problemáticas:

Kick Sauber: mAP50 ~0.619
Alpine: mAP50 ~0.556
Williams: mAP50 ~0.487
Resultados del Nuevo Modelo Fine-Tuned:
En Train:
mAP50: 0.840
mAP50-95: 0.728
Precisión: 0.643
Recall: 0.761
Clases Problemáticas:

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.522
En Validación:
mAP50: 0.839
mAP50-95: 0.729
Precisión: 0.643
Recall: 0.762
Clases Problemáticas:

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.519
Observaciones:
El mAP50 ha mejorado notablemente, subiendo de 0.772 a 0.840 en entrenamiento y 0.839 en validación.
El mAP50-95 también sube de 0.449 a 0.728 en entrenamiento y 0.729 en validación, lo cual indica un aumento en la precisión en diferentes umbrales de IoU.
La precisión ha subido ligeramente de 0.733 a 0.643 en el entrenamiento, pero con un recall mucho más alto, lo que sugiere que el modelo fine-tuned ha mejorado la capacidad de identificar los objetos relevantes, aunque el número de falsos positivos puede haber aumentado.
Las clases problemáticas como Kick Sauber, Alpine, y Williams han mostrado mejoras en mAP50, con un aumento significativo en Alpine (0.928 en validación, frente a 0.556 anteriormente).
Conclusión:
El modelo fine-tuned muestra una mejora general significativa, especialmente en mAP50 y en la capacidad para manejar clases problemáticas, aunque el modelo podría beneficiarse de un ajuste adicional en la precisión, especialmente para clases como Williams. Reducir las epochs de 40 a menos podría funcionar si se optimizan más hiperparámetros, pero parece que las 21 epochs han sido suficientes. 


### Siguiente modelo: peso de 2 a williams en el yaml.

Resultados del Nuevo Modelo (Con Peso para Williams):
mAP50: 0.840 en entrenamiento, 0.839 en validación.
mAP50-95: 0.728 en entrenamiento, 0.729 en validación.
Precisión: 0.643 en entrenamiento, 0.643 en validación.
Recall: 0.761 en entrenamiento, 0.762 en validación.
Resultados Anteriores (Sin Ajuste):
mAP50: 0.772 en entrenamiento, 0.772 en validación.
mAP50-95: 0.449 en entrenamiento, 0.449 en validación.
Precisión: 0.733 en entrenamiento, 0.733 en validación.
Recall: 0.615 en entrenamiento, 0.615 en validación.
Clases Problemáticas:
Antes (sin ajuste):

Kick Sauber: mAP50 ~0.619
Alpine: mAP50 ~0.556
Williams: mAP50 ~0.487
Ahora (con ajuste):

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.522
Observaciones:
mAP50 ha mejorado significativamente: de 0.772 a 0.840 en entrenamiento y de 0.772 a 0.839 en validación, lo que refleja un aumento en la capacidad de detectar objetos de manera precisa.
mAP50-95 también ha mejorado considerablemente, subiendo de 0.449 a 0.728 en entrenamiento y 0.449 a 0.729 en validación.
Precisión y Recall en general han subido, pero la precisión ha tenido un ligero descenso (0.733 a 0.643), lo cual podría indicar un mayor número de falsos positivos, aunque el recall ha aumentado, lo que indica una mejora en la identificación de objetos relevantes.
Williams sigue siendo un reto con un mAP50 de 0.522 en entrenamiento y 0.519 en validación, lo que es una ligera mejora respecto al valor anterior de 0.487, pero sigue estando por debajo de otros equipos.

In [5]:
# Cargar el modelo entrenado para hacer predicciones
model_fine_tuned_test = YOLO("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")
model_fine_tuned_test.to(DEVICE)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0-1): 2 x Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=Tr

In [6]:
# Evaluar en el conjunto de prueba
test_results = model_fine_tuned_test.predict(source="../f1-strategy/f1-dataset/test/images", save=True)

image 1/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1-2023-Brazilian-GP-FP1-21_jpg.rf.e0a6d6d14a2b145d1285111d65dcf969.jpg: 640x640 1 Ferarri, 14.4ms
image 2/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1019924227-LAT-20231105-GP2320_143500SMG_2193-747_jpg.rf.cd24225e35180d6a3ede83dfb291abea.jpg: 640x640 3 Williamss, 17.0ms
image 3/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\13-sunday-mexicocity-2023_jpg.rf.9fb9bf359354501321e3dbd03e75d0d6.jpg: 640x640 1 Aston Martin, 14.3ms
image 4/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\Ferraritop_jpg.rf.2986765a7cdb43c522f47262e9e4b1ab.jpg: 640x640 1 Ferarri, 13.5ms
image 5/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Fina

In [7]:
# Ver el contenido de test_results
print(test_results)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Kick Sauber', 1: 'Racing Bulls', 2: 'Alpine', 3: 'Aston Martin', 4: 'Ferarri', 5: 'Haas', 6: 'Mclaren', 7: 'Mercedes', 8: 'Red Bull', 9: 'Williams'}
obb: None
orig_img: array([[[ 32,  39,  12],
        [ 35,  42,  15],
        [ 47,  52,  25],
        ...,
        [233, 220, 222],
        [233, 220, 222],
        [233, 220, 222]],

       [[ 30,  37,  10],
        [ 32,  39,  12],
        [ 42,  47,  20],
        ...,
        [234, 221, 223],
        [233, 220, 222],
        [233, 220, 222]],

       [[ 26,  33,   6],
        [ 28,  35,   8],
        [ 36,  40,  15],
        ...,
        [234, 221, 223],
        [234, 221, 223],
        [233, 220, 222]],

       ...,

       [[ 29,  88,  67],
        [ 27,  89,  65],
        [ 27,  89,  67],
        ...,
        [136, 145, 154],
        [135, 144, 153],
        [134, 143, 152]],

       [[ 

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv
import torch

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Cargar modelo fine-tuned
model = YOLO("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")
model.to(DEVICE)

# Configurar el anotador (sólo para las cajas)
box_annotator = sv.BoxAnnotator(thickness=2)

def shrink_boxes(boxes, shrink_ratio=0.8):
    # boxes: lista o array de [x1, y1, x2, y2]
    new_boxes = []
    for box in boxes:
        x1, y1, x2, y2 = box
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        w = (x2 - x1) * shrink_ratio
        h = (y2 - y1) * shrink_ratio
        new_box = [cx - w/2, cy - h/2, cx + w/2, cy + h/2]
        new_boxes.append(new_box)
    # Convertir a numpy array para que se pueda aplicar astype(int) en el anotador.
    return np.array(new_boxes)

def detect_in_video(model, video_path):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionar el frame a 640x640 para que coincidan las coordenadas
        frame_resized = cv2.resize(frame, (640, 640))
        
        # Ejecutar la predicción ajustando los parámetros de confianza e IoU
        results = model.predict(frame_resized, conf=0.5, iou=0.35, verbose=False)[0]
        
        # Convertir las detecciones a un objeto de supervision
        detections = sv.Detections.from_ultralytics(results)
        
        # Filtrar detecciones: conservar solo aquellas con confianza >= 0.4
        mask = detections.confidence >= 0.5
        detections.xyxy = detections.xyxy[mask]
        detections.confidence = detections.confidence[mask]
        detections.class_id = detections.class_id[mask]
        
        # Generar etiquetas (nombre del equipo y confianza) para cada detección
        labels = []
        for i in range(len(detections)):
            conf = detections.confidence[i]
            class_id = detections.class_id[i]
            label = f"{model.model.names[int(class_id)]} {conf:.2f}"
            labels.append(label)

            team = model.model.names[int(class_id)]

            # Filtrar detecciones erróneas de Williams
            if team == "Williams" and conf < 0.7:
                continue  

            
            labels.append(label)
        # Asignar las etiquetas al objeto detections
        detections.labels = labels
        
        # Reducir el tamaño de las bounding boxes y convertir a array de NumPy
        detections.xyxy = shrink_boxes(detections.xyxy, shrink_ratio=0.9)
        
        # Anotar el frame con las cajas (sobre el frame redimensionado)
        annotated_frame = box_annotator.annotate(
            scene=frame_resized.copy(),
            detections=detections
        )
        
        # Añadir manualmente los textos (nombre del equipo) sobre cada caja
        for i, box in enumerate(detections.xyxy):
            x1, y1, x2, y2 = map(int, box)
            cv2.putText(
                annotated_frame,
                labels[i],
                (x1, max(y1 - 10, 0)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
                cv2.LINE_AA
            )
            
        cv2.imshow('F1 Detection', annotated_frame)
        # Si se presiona "q", se detiene la ejecución
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Detección detenida por el usuario.")
            break

    cap.release()
    cv2.destroyAllWindows()

# Ejecutar la detección en tiempo real a partir de un vídeo
detect_in_video(model, "../f1-strategy/data/videos/spain_2023_race.mp4.f299.mp4")


Detección detenida por el usuario.
